In [0]:
import sys
import time
import requests
import json
import pyspark.sql.functions as F
import pandas as pd

In [0]:
auth_key = '' #Assembly AI API Key

In [0]:
#creating dataframe for all the records received today
new_file_logs  = spark.sql('SELECT patient_id, appointment_id, index_id, audio_record_file_url FROM cyb.silver_data WHERE index_id NOT IN (SELECT DISTINCT(index_id) FROM cyb.transcript_main)').toPandas()

In [0]:
# dbutils.fs.ls('/mnt/cyberclinic/Transcript_assemblyai')

Out[12]: [FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1009271302.json', name='1009271302.json', size=617456),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1009860567.json', name='1009860567.json', size=144),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1013630930.json', name='1013630930.json', size=446618),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1015194392.json', name='1015194392.json', size=116524),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1020357526.json', name='1020357526.json', size=827653),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1030518210.json', name='1030518210.json', size=830015),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1032666590.json', name='1032666590.json', size=1285),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1035683924.json', name='1035683924.json', size=778011),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1037894457.json', name='1037894457.json', size=42321),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1040830776.json', name='1040830776.json', size=648881),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1041427.json', name='1041427.json', size=330902),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1051987338.json', name='1051987338.json', size=589416),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1056724303.json', name='1056724303.json', size=231099),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1058275426.json', name='1058275426.json', size=783743),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1060750754.json', name='1060750754.json', size=446784),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1064229559.json', name='1064229559.json', size=581906),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1067008866.json', name='1067008866.json', size=28061),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1072223899.json', name='1072223899.json', size=808902),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1072589537.json', name='1072589537.json', size=647760),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/109185615.json', name='109185615.json', size=860165),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1106093817.json', name='1106093817.json', size=544555),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1106631548.json', name='1106631548.json', size=526701),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1106868281.json', name='1106868281.json', size=615),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1113478469.json', name='1113478469.json', size=641373),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/112021928.json', name='112021928.json', size=12519),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1123991999.json', name='1123991999.json', size=710981),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1126764662.json', name='1126764662.json', size=904335),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1128058918.json', name='1128058918.json', size=922950),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1132110593.json', name='1132110593.json', size=635051),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1139531186.json', name='1139531186.json', size=601741),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1140858236.json', name='1140858236.json', size=629998),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1146251868.json', name='1146251868.json', size=513634),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1149017278.json', name='1149017278.json', size=169664),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1151985248.json', name='1151985248.json', size=144),
 FileInfo(path='dbfs:/mnt/cyberclinic/Transcript_assemblyai/1157189635.

In [0]:
# do code for comparing with dumps
ids = [f.name[:-5] for f in dbutils.fs.ls('/mnt/cyberclinic/Transcript_assemblyai')]

In [0]:
app_res_list = []
for i in range(0,len(new_file_logs)):
  if str(new_file_logs.index_id[i]) in ids:
    continue
  audio_url = new_file_logs.audio_record_file_url[i]
  endpoint = "https://api.assemblyai.com/v2/transcript"
  json_ = {
  "audio_url": audio_url,
  "speaker_labels": True,
  }
  headers_ = {
    "authorization": auth_key,
    "content-type": "application/json"
  }
  response = requests.post(endpoint, json=json_, headers=headers_)
  tmp = {'appointment_id': str(new_file_logs.appointment_id[i]),
         'patient_id' : str(new_file_logs.patient_id[i]),
         'index_id' : str(new_file_logs.index_id[i]),
         'transcript_id': response.json()['id'],
         'status': response.json()['status']
        }
  app_res_list.append(tmp)

{"error": "Your current account balance is negative. Please top up to continue using the API."}

--------------------------------------------------------------------------- 
 KeyError Traceback (most recent call last)
 <command-4458331744327786> in <module> 
 18 'patient_id' : str ( new_file_logs . patient_id [ i ] ) , 
 19 'index_id' : str ( new_file_logs . index_id [ i ] ) , 
 ---> 20 'transcript_id' : response . json ( ) [ 'id' ] , 
 21 'status' : response . json ( ) [ 'status' ] 
 22 }

 KeyError : 'id'

In [0]:
transcript_list = []
error_list = []
dump_json_file = []
c = len(app_res_list)
while c :
  time.sleep(300)
  for i in range(len(app_res_list)):
    if app_res_list[i]['status'] == "#":
      continue
    status_point = 'https://api.assemblyai.com/v2/transcript/' + str(app_res_list[i]['transcript_id'])
    status_header = {'authorization':auth_key} 
    status_check = requests.get(status_point, headers=status_header)
    if not status_check.ok:
      app_res_list[i]['status'] = '#'
      continue
    ts = status_check.json()
    if ts['status'] == 'completed':
      app_res_list[i]['status'] = '#'
      c -= 1
      tmp = app_res_list[i].copy()
      tmp['transcript'] = ts['utterances']
      to_be_added = {
        "index_id":tmp['index_id'],
        "patient_id":tmp['patient_id'],
        "appointment_id":tmp['appointment_id'],
        "assmb_response":ts
      }
      dump_json_file.append(to_be_added)
      transcript_list.append(tmp)
    if ts['status'] == 'error':
      app_res_list[i]['status'] = '#'
      c -= 1

#       tmp = app_res_list[i].copy()
#       tmp['transcript_json'] = ts
#       error_list.append(tmp)

In [0]:
for t in dump_json_file:
  file_name = t['index_id']
  with open(f"/dbfs/mnt/cyberclinic/Transcript_assemblyai/{file_name}.json","w") as f:
    json.dump(t,f)

In [0]:
DF = pd.DataFrame()
for t in transcript_list:
  spk1Turns = -1
  spk2Turns = -1
  first_flg = 0
  spk1 = []
  spk2 = []
  spkFlag = 0
  spk1_word_cnt = 0
  spk2_word_cnt = 0
  spk1_quest_cnt = 0
  spk2_quest_cnt = 0
  if t['transcript'] == []:
      print('Empty file : ',t['appointment_id'])
      continue
  for phrase in t['transcript']:
    if first_flg == 0:
      first_spk = phrase['speaker']
      first_flag = 1
    if phrase['speaker'] == 'A':
      if spkFlag != phrase['speaker']:
        spkFlag = phrase['speaker']
        spk1Turns += 1
        spk1.insert(spk1Turns,phrase['text'])
      else:
        spk1[spk1Turns] = spk1[spk1Turns] + " " + phrase['text']
      spk1_word_cnt += len(phrase['text'].split())
      for i in phrase['text']:
        if i == '?':
          spk1_quest_cnt += 1
    if phrase['speaker'] == "B":
      if spkFlag != phrase['speaker']:
        spkFlag = phrase['speaker']
        spk2Turns += 1
        spk2.insert(spk2Turns,phrase['text'])
      else:
        spk2[spk2Turns] = spk2[spk2Turns] + ' ' + phrase['text']
      spk2_word_cnt += len(phrase['text'].split())
      for i in phrase['text']:
        if i == '?':
          spk2_quest_cnt += 1
  if((spk2_quest_cnt > spk1_quest_cnt) and (first_spk == "B")):
    data = {'Patient':spk1, 'Practitioner':spk2[1:]}
  elif((spk2_quest_cnt > spk1_quest_cnt) and (first_spk == "A")):
    data = {'Patient':spk1, 'Practitioner':spk2}
  elif((spk2_quest_cnt < spk1_quest_cnt) and (first_spk == "B")):
    data = {'Patient':spk2, 'Practitioner':spk1}
  elif((spk2_quest_cnt < spk1_quest_cnt) and (first_spk == "A")):
    data = {'Patient':spk2, 'Practitioner':spk1[1:]}
  else:
    data = {'Patient':"", 'Practitioner':""}
  if len(data['Patient'])>0 and len(data['Practitioner'])>0:
    df=pd.DataFrame({ key:pd.Series(value) for key, value in data.items()})
    df['appointment_id'] = t['appointment_id']
    df['patient_id']= t['patient_id']
    df['index'] = [*range(1,1+len(df))]
    DF=pd.concat([DF,df])

In [0]:
if len(DF):
  df1 = spark.createDataFrame(DF)
  df1 = (df1.withColumn('patient_id',F.col('patient_id').cast('long'))
         .withColumn('appointment_id',F.col('appointment_id').cast('long'))
        )
  (df1.select('index','patient_id','appointment_id','Practitioner','Patient')
   .write
   .format('delta')
   .mode('append')
   .saveAsTable('cyb.transcript_main')
  )